# Ceneo Scraper

## Imports

In [42]:
import os
import json
import requests
from bs4 import  BeautifulSoup

## Dictionary with Opinion structure

In [43]:
selectors= { 
"opinion_id": (None,'data-entry-id'), 
"author": ("span.user-post__author-name",), 
"authors_recommendation": ("span.user-post__author-recomendation > em",), 
"stars": ("span.user-post__score-count",), 
"content": ("div.user-post__text",), 
"pros": ("div.review-feature__title--positives ~ div.review-feature__item",None , True), 
"cons": ("div.review-feature__title--negatives ~ div.review-feature__item", None, True), 
"helpful": ("button.vote-yes > span",), 
"unhelpful": ("button.vote-no > span",), 
"publish_date": ("span.user-post__published > time:nth-child(1)","datetime"), 
"purchase_date": ("span.user-post__published > time:nth-child(2)","datetime"), 
}

## Transformation function

In [44]:
def score (score:str) -> float:
    s = score.split("/")
    return float(s[0]).replace(",",".")/float(s[1])

## Dictionary with transformations


In [45]:
transformations = {
    "recommendation":lambda r: True if r == "Polecam" else False if r == "Nie polecam" else None,
    "score": score,
    "helpful": int,
    "unhelpful": int
}

# Send request to Ceneo.pl service

In [46]:
product_id = "108290707"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews_scroll"



## Convert plain text HTML code to DOM structure

## Components of single opinion

|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion’s author|span.user-post__author-name|author|
|author’s recommendation|span.user-post__author-recomendation em|recommendation|
|score expressed in number of stars|span.user-post__score-count|score|
|opinion’s content|div.user-post__text|content|
|list of product advantages|div.review-feature--positives ~ div.review-feature__item|advantages|
|list of product disadvantages|div.review-feature--negatives ~ div.review-feature__item |disadvantages|
|how many users think that opinion was helpful|button.vote-yes > span|helpful|
|how many users think that opinion was unhelpful|button.vote-no > span|not_helpful|
|publishing date|span.user-post__published > time:nth-child(1)["datetime"]|pub_date|
|purchase date|span.user-post__published > time:nth-child(2)["datetime"]|pur_date|



In [47]:
def extract_content(ancestor, selector, attribute=None, return_list=False):
    if selector:
        if return_list:
            if attribute:
                return [tag[attribute].strip() for tag in ancestor.select(selector)]
            return [tag.text.strip() for tag in ancestor.select(selector)]
        if attribute:
            try:
                return opinion.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.text.strip()


## Extract all opinions about product

In [48]:
all_opinions=[] 
while(url): 
    response = requests.get(url) 
    response.status_code 
    page_dom = BeautifulSoup(response.text, "html.parser") 
    opinions = page_dom.select("div.js_product-review") 
    for opinion in opinions: 
        single_opinion = { 
            key: extract_content(opinion, *value) 
                for key, value in selectors.items() 
        } 
        for key, value in transformations.items():
            sigle_opinion[key] = value(sigle_opinion[key])
        all_opinions.append(single_opinion)
    try: 
        url = "https://www.ceneo.pl"+extract_content(page_dom,"a.pagination__next","href") 
    except TypeError: 
        url = None

NameError: name 'sigle_opinion' is not defined

In [ ]:
if not os.path.exists("opinions"): 
    os.mkdir("opinions") 
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf: 
    json.dump(all_opinions, jf, indent=4 , ensure_ascii=False)
